In [11]:
def load_grid(file_path):
    """Reads the grid from the input file and returns it as a list of strings."""
    with open(file_path) as file:
        return file.read().strip().split()

def find_start_position(grid):
    """Finds the starting position 'S' in the grid and returns its coordinates as a set."""
    return {(row.index("S"), row_index) for row_index, row in enumerate(grid) if "S" in row}

def is_move_valid(grid, current_position, next_position, visited_positions):
    """Checks if a move from current_position to next_position is valid based on pipe connectivity and bounds."""
    current_x, current_y = current_position
    next_x, next_y = next_position

    # Ensure the next position is within the grid bounds
    if not (0 <= next_y < len(grid) and 0 <= next_x < len(grid[next_y])):
        return False

    # Ensure the next position has not been visited yet
    if next_position in visited_positions:
        return False

    # Define valid pipe transitions based on direction of movement
    direction_transitions = {
        (1, 0): ("-LFS", "-J7"),   # Move right
        (-1, 0): ("-J7S", "-LF"),  # Move left
        (0, 1): ("|F7S", "|LJ"),   # Move down
        (0, -1): ("|LJS", "|F7"),  # Move up
    }

    delta_x, delta_y = next_x - current_x, next_y - current_y

    # Check if the move is valid according to the pipe types
    if (delta_x, delta_y) in direction_transitions:
        valid_current_chars, valid_next_chars = direction_transitions[(delta_x, delta_y)]
        return grid[current_y][current_x] in valid_current_chars and grid[next_y][next_x] in valid_next_chars

    return False

def explore_pipe_loop(grid):
    """Performs BFS to explore the pipe loop from the start position, finds the farthest point, and calculates enclosed area."""
    start_positions = find_start_position(grid)
    visited_positions = set()
    exploration_queue = [start_positions]

    # Perform BFS to explore the grid
    while exploration_queue[-1]:
        current_layer = exploration_queue[-1] - visited_positions
        visited_positions |= current_layer

        next_layer = {
            (next_x, next_y) for (current_x, current_y) in current_layer
            for next_x, next_y in [(current_x + 1, current_y), (current_x - 1, current_y), (current_x, current_y + 1), (current_x, current_y - 1)]
            if is_move_valid(grid, (current_x, current_y), (next_x, next_y), visited_positions)
        }

        exploration_queue.append(next_layer)

    # Calculate the distance to the farthest point
    max_steps = len(exploration_queue) - 2

    # Calculate the number of tiles enclosed by the loop
    enclosed_area = sum(
        sum(
            grid[row][col] in "|JLS"
            for col in range(width) if (col, row) in visited_positions
        ) % 2
        for row in range(len(grid)) for width in range(len(grid[row]))
        if (width, row) not in visited_positions
    )

    return max_steps, enclosed_area

def main():
    """Main function to orchestrate loading the grid, exploring the pipe loop, and outputting the results."""
    grid = load_grid("/content/AoC_2023_Day10.txt")
    max_steps, enclosed_area = explore_pipe_loop(grid)
    print(f"Maximum steps to the farthest point: {max_steps}")
    print(f"Number of tiles enclosed by the loop: {enclosed_area}")

if __name__ == "__main__":
    main()


Maximum steps to the farthest point: 6733
Number of tiles enclosed by the loop: 435
